In [1]:
import pandas as pd 
import numpy as np


In [2]:
!pip install pypdf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ramya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from pypdf import PdfReader
import os

In [4]:
def extract_text_from_pdfs(pdf_folder):
    """
    Extracts text from all PDFs in a given folder and returns a Pandas DataFrame.

    Args:
        pdf_folder (str): Path to the folder containing PDF files.

    Returns:
        pd.DataFrame: A DataFrame containing extracted text with columns: 
                      ["File Name", "Page Number", "Text"]
    """
    all_data = []

    # List all PDF files in the given folder
    pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)
        reader = PdfReader(pdf_path)

        # Extract text from each page
        for page_number, page in enumerate(reader.pages):
            text = page.extract_text()
            if text:  # Avoid empty pages
                all_data.append([pdf_file, page_number + 1, text])

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=["File Name", "Page Number", "Text"])
    return df

# Example Usage:
pdf_folder = "research_papers"  # Replace with the actual folder path
df = extract_text_from_pdfs(pdf_folder)

# Display the extracted text
df.to_csv("extracted_text.csv", index=False)
print(df)


                                            File Name  Page Number  \
0                                    2305.05422v1.pdf            1   
1                                    2305.05422v1.pdf            2   
2                                    2305.05422v1.pdf            3   
3                                    2305.05422v1.pdf            4   
4                                    2305.05422v1.pdf            5   
5                                    2305.05422v1.pdf            6   
6                                    2305.05422v1.pdf            7   
7                                    2305.05422v1.pdf            8   
8                                    2305.05422v1.pdf            9   
9                                    2305.05422v1.pdf           10   
10  A_Conceptual_Model_for_Implementing_Explainabl...            1   
11  A_Conceptual_Model_for_Implementing_Explainabl...            2   
12  A_Conceptual_Model_for_Implementing_Explainabl...            3   
13  A_Conceptual_Mod

In [5]:
df.head(20)

,File Name,Page Number,Text
0,2305.05422v1.pdf,1,L. Erculiani et al. /\nEgocentric Hierarchical...
1,2305.05422v1.pdf,2,L. Erculiani et al. /\nMusical instrumentsG: d...
2,2305.05422v1.pdf,3,L. Erculiani et al. /\nv1\n v2\n v3\nE\nFigure...
3,2305.05422v1.pdf,4,L. Erculiani et al. /\nAlgorithm 1The main loo...
4,2305.05422v1.pdf,5,L. Erculiani et al. /\nAlgorithm 2The procedur...
5,2305.05422v1.pdf,6,L. Erculiani et al. /\ncurrentGenus candidateG...
6,2305.05422v1.pdf,7,L. Erculiani et al. /\nFigure 5. Comparison be...
7,2305.05422v1.pdf,8,L. Erculiani et al. /\n5. Related work\nOur wo...
8,2305.05422v1.pdf,9,L. Erculiani et al. /\nReferences\n[1] Kuang Z...
9,2305.05422v1.pdf,10,"L. Erculiani et al. /\n[28] Fu Y , Dong H, Ma ..."


In [6]:
# Data preprocessing
df.describe()

,Page Number
count,46.000000
mean,5.826087
std,3.535875
min,1.000000
25%,3.000000
50%,5.500000
75%,8.000000
max,14.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   File Name    46 non-null     object
 1   Page Number  46 non-null     int64 
 2   Text         46 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [8]:
# check for null values
df.isnull().sum()

File Name      0
Page Number    0
Text           0
dtype: int64

In [9]:
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ramya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
!pip install spacy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ramya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import nltk
import string
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import spacy

In [12]:
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 495.5 kB/s eta 0:00:26
     --------------------------------------- 0.1/12.8 MB 819.2 kB/s eta 0:00:16
     - -------------------------------------- 0.4/12.8 MB 2.4 MB/s eta 0:00:06
     --- ------------------------------------ 1.3/12.8 MB 5.8 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 7.9 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 8.8 MB/s eta 0:00:02
     ----------- ---------------------------- 3.8/12.8 MB 10.6 MB/s eta 0:00:01
     -------------- ------------------------- 4.6/12.8 MB 11.7 MB/s eta 0:00:01
     ----------------- ---------------------- 5.7/12.8 MB 13.4 MB/s eta 0:00:01
     -----------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\ramya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ramya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [14]:
def text_lowering(text):

    return text.lower()

def clean_text_from_unwanted_characters(text):
    """
    Removes unwanted characters and punctuation from the given text.

    Parameters:
    text (str): The input text to be cleaned.

    Returns:
    str: The cleaned text without unwanted characters.
    """
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

def lemmatize_text(text):

    # Tokenize the text using spaCy
    doc = nlp(text)
    # Lemmatize each token and join them back into a string
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

def tokenize_word(text):

    return word_tokenize(text)

def remove_stopwords(tokenized_Text):

    # Define a list to store non-stopwords
    filtered_Text = []
    # Loop through each token in the tokenized Text
    for token in tokenized_Text:
        # Check if the token is not a stopword
        if token not in stop_words:
            # If not a stopword, add it to the filtered list
            filtered_Text.append(token)

    return filtered_Text

def extract_final_data_as_data(file_name, df):
    """
    Writes the given DataFrame to a CSV file.

    Parameters:
    file_name (str): The name of the output CSV file.
    df (DataFrame): The DataFrame to be written to the CSV file.
    """
    df.to_csv(file_name)


def join_as_sentence(word_list):
    """
    Joins a list of words into a sentence.

    Parameters:
    word_list (list): List of words to be joined.

    Returns:
    str: The joined sentence.
    """
    return ' '.join(word_list)


In [15]:

def main(input_file_name, output_file_name):
    print("Reading data...")
    df = pd.read_csv(f"{input_file_name}.csv")


    # Selecting required columns
    required_column_names = ["File Name","Page Number","Text"]
    df = df[required_column_names]

    # Dropping rows with missing values
    print("Dropping rows with missing values...")
    df = df.dropna()

    print("Preprocessing text data...")
    # Lowercasing the Texts
    df['Lowered_Text'] = df['Text'].apply(text_lowering)

    # Removing unwanted characters
    df['Punctuation_removed_Text'] = df['Lowered_Text'].apply(clean_text_from_unwanted_characters)

    # Lemmatizing the Texts
    df['lemmatized_Text'] = df['Punctuation_removed_Text'].apply(lemmatize_text)

    # Tokenizing the Texts
    df['Tokenized_Text'] = df['lemmatized_Text'].apply(tokenize_word)

    # Removing stopwords
    df['stopwords_removed_tokenized_Text'] = df['Tokenized_Text'].apply(remove_stopwords)

    df['final_cleaned_Text_joined'] = df['stopwords_removed_tokenized_Text'].apply(join_as_sentence)

    print("Extracting final data...")
    extract_final_data_as_data(output_file_name, df)


    print("Processing completed.")


In [19]:
#input file name
input_file_name = "extracted_text"
#output file name
output_file_name = f"Text_preprocessed.csv"

main(input_file_name,output_file_name)

Reading data...
Dropping rows with missing values...
Preprocessing text data...
Extracting final data...
Processing completed.


In [21]:
new_df = pd.read_csv("Text_preprocessed.csv")
new_df.head(20)

,Unnamed: 0,File Name,Page Number,Text,Lowered_Text,Punctuation_removed_Text,lemmatized_Text,Tokenized_Text,stopwords_removed_tokenized_Text,final_cleaned_Text_joined
0,0,2305.05422v1.pdf,1,L. Erculiani et al. /\nEgocentric Hierarchical...,l. erculiani et al. /\negocentric hierarchical...,l erculiani et al \negocentric hierarchical vi...,l erculiani et al \n egocentric hierarchical v...,"['l', 'erculiani', 'et', 'al', 'egocentric', '...","['l', 'erculiani', 'et', 'al', 'egocentric', '...",l erculiani et al egocentric hierarchical visu...
1,1,2305.05422v1.pdf,2,L. Erculiani et al. /\nMusical instrumentsG: d...,l. erculiani et al. /\nmusical instrumentsg: d...,l erculiani et al \nmusical instrumentsg devic...,l erculiani et al \n musical instrumentsg devi...,"['l', 'erculiani', 'et', 'al', 'musical', 'ins...","['l', 'erculiani', 'et', 'al', 'musical', 'ins...",l erculiani et al musical instrumentsg device ...
2,2,2305.05422v1.pdf,3,L. Erculiani et al. /\nv1\n v2\n v3\nE\nFigure...,l. erculiani et al. /\nv1\n v2\n v3\ne\nfigure...,l erculiani et al \nv1\n v2\n v3\ne\nfigure 2 ...,l erculiani et al \n v1 \n v2 \n v3 \n e \n ...,"['l', 'erculiani', 'et', 'al', 'v1', 'v2', 'v3...","['l', 'erculiani', 'et', 'al', 'v1', 'v2', 'v3...",l erculiani et al v1 v2 v3 e figure 2 example ...
3,3,2305.05422v1.pdf,4,L. Erculiani et al. /\nAlgorithm 1The main loo...,l. erculiani et al. /\nalgorithm 1the main loo...,l erculiani et al \nalgorithm 1the main loop o...,l erculiani et al \n algorithm 1the main loop ...,"['l', 'erculiani', 'et', 'al', 'algorithm', '1...","['l', 'erculiani', 'et', 'al', 'algorithm', '1...",l erculiani et al algorithm 1the main loop fra...
4,4,2305.05422v1.pdf,5,L. Erculiani et al. /\nAlgorithm 2The procedur...,l. erculiani et al. /\nalgorithm 2the procedur...,l erculiani et al \nalgorithm 2the procedure p...,l erculiani et al \n algorithm 2the procedure ...,"['l', 'erculiani', 'et', 'al', 'algorithm', '2...","['l', 'erculiani', 'et', 'al', 'algorithm', '2...",l erculiani et al algorithm 2the procedure pre...
5,5,2305.05422v1.pdf,6,L. Erculiani et al. /\ncurrentGenus candidateG...,l. erculiani et al. /\ncurrentgenus candidateg...,l erculiani et al \ncurrentgenus candidategenu...,l erculiani et al \n currentgenus candidategen...,"['l', 'erculiani', 'et', 'al', 'currentgenus',...","['l', 'erculiani', 'et', 'al', 'currentgenus',...",l erculiani et al currentgenus candidategenuse...
6,6,2305.05422v1.pdf,7,L. Erculiani et al. /\nFigure 5. Comparison be...,l. erculiani et al. /\nfigure 5. comparison be...,l erculiani et al \nfigure 5 comparison betwee...,l erculiani et al \n figure 5 comparison betwe...,"['l', 'erculiani', 'et', 'al', 'figure', '5', ...","['l', 'erculiani', 'et', 'al', 'figure', '5', ...",l erculiani et al figure 5 comparison naive mo...
7,7,2305.05422v1.pdf,8,L. Erculiani et al. /\n5. Related work\nOur wo...,l. erculiani et al. /\n5. related work\nour wo...,l erculiani et al \n5 related work\nour work i...,l erculiani et al \n 5 relate work \n our work...,"['l', 'erculiani', 'et', 'al', '5', 'relate', ...","['l', 'erculiani', 'et', 'al', '5', 'relate', ...",l erculiani et al 5 relate work work implement...
8,8,2305.05422v1.pdf,9,L. Erculiani et al. /\nReferences\n[1] Kuang Z...,l. erculiani et al. /\nreferences\n[1] kuang z...,l erculiani et al \nreferences\n1 kuang z yu j...,l erculiani et al \n reference \n 1 kuang z yu...,"['l', 'erculiani', 'et', 'al', 'reference', '1...","['l', 'erculiani', 'et', 'al', 'reference', '1...",l erculiani et al reference 1 kuang z yu j li ...
9,9,2305.05422v1.pdf,10,"L. Erculiani et al. /\n[28] Fu Y , Dong H, Ma ...","l. erculiani et al. /\n[28] fu y , dong h, ma ...",l erculiani et al \n28 fu y dong h ma yf zhan...,l erculiani et al \n 28 fu y dong h ma yf zh...,"['l', 'erculiani', 'et', 'al', '28', 'fu', 'y'...","['l', 'erculiani', 'et', 'al', '28', 'fu', 'do...",l erculiani et al 28 fu dong h yf zhang z xue ...
